## 2 кейс

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [1]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2026-02-20 04:08:49--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 140.82.114.4
Connecting to gist.github.com (gist.github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2026-02-20 04:08:49--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json’

creds.json          100%[===================>]   2.30K  --.-KB/s    in 0s      

2026-02-20 04:08:49 (38.4 MB/s) - ‘creds.json’ saved [2

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Небольшое описание столбцов в таблицах:  
**Лист1:** `student_id` - айди студента; `student_name` - имя студента; `installment` (Y/N) - факт наличия рассрочки у студента (Y - рассрочка есть, N - рассрочки нет).  
**Лист2:** `student_id` - айди студента; `last_payment_date` - дата последней полученной оплаты; `expected_payment_date` - дата, в которую ожидаем следующий платеж (рассчитывается от даты последнего платежа).  
**Лист3:** `student_id` - айди студента; `already_payed_amount` - уже выплаченная часть рассрочки; `left_to_pay` - оставшаяся (невыплаченная) часть; `one-time_payment` - единоразовый платеж; `installment_amount` - общая сумма, которая бралась в рассрочку.

Как примерно должен выглядеть результат:

In [ ]:
with open('result.txt') as f:
  print(f.read())

Студент Иванов У.У. - долг 100000 рублей
Студент Петров Е.Е. - долг 250000 рублей
Студент Сидоров И.И. - долг 10000 рублей


In [2]:
#@title Если возникнут трудности с определением `scope`, подсказка:

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев** (183 дня).

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023). При расчетах отталкивайтесь от даты ожидаемого платежа, разницу между платежами берите 183 дня.

**Решение**

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Указываем необходимые права доступа к таблицам
scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']
# Загружаем ключи аутентификации из файла json
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)

# Авторизуемся в Google Sheets API
client = gspread.authorize(creds)

sheet_names = client.open("Installments").worksheet("Лист1")
sheet_dates = client.open("Installments").worksheet("Лист2")
sheet_pays = client.open("Installments").worksheet("Лист3")


In [57]:
from datetime import datetime

def generate_report(sheet_names, sheet_dates, sheet_pays):
  data_names = sheet_names.get_all_records()
  data_dates = sheet_dates.get_all_records()
  data_pays = sheet_pays.get_all_records()

  # Меняем структуру данных:
  # d_names = {id : names},
  # d_dates = {id : [last_pay, expected_pay]}
  # d_pays = {id : [left_to_pay, one-time_pay]}
  d_names = {}
  for dic in data_names:
    d_names[dic['student_id']] = dic['student_name']

  d_dates = {}

  for dic in data_dates:
    d_dates[dic['student_id']] = [dic['last_payment_date'], dic['expected_payment_date']]

  d_pays = {}
  for dic in data_pays:
    d_pays[dic['student_id']] =[dic['left_to_pay'], dic['one-time_payment']]

  # Вводим 'сегодняшнюю' дату
  today = datetime.strptime('01.03.2023', '%d.%m.%Y').date()

  # Записываем список должников в файл student_debt_report.txt
  with open('student_debt_report.txt', 'w') as f_out:
    for id, val in d_dates.items():
      last_pay = datetime.strptime(val[0], '%d.%m.%Y').date()
      expected_pay = datetime.strptime(val[1], '%d.%m.%Y').date()
      if (last_pay - expected_pay).days < 0 | (today -  expected_pay).days > 0:
        N = 1 + (today - expected_pay).days // 183
        debt = min(N * d_pays[id][1], d_pays[id][0])
        w_str = f'Студент {d_names[id]} - долг {debt} рублей\n'
        f_out.write(w_str)
        #print(f'Студент {id}: {d_names[id]} - долг {N * d_pays[id][1]},{d_pays[id][0]} рублей')

generate_report(sheet_names, sheet_dates, sheet_pays)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [58]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd

user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')

--2026-02-20 06:24:44--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 140.82.112.3
Connecting to gist.github.com (gist.github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2026-02-20 06:24:44--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: ‘student_debt.txt’

student_debt.txt    100%[===================>]  11.06K  --.-KB/s    in 0s      

2026-02-20 06:24:44 (90.1 MB/s)

In [59]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
